# Convolutional Neural Network for Urban Sound Classification using MFCCs



In this example we apply a CNN to the Urban Sound Classification problem.

A network with three convolutional layers and two dense layers of moderate size achieves 90% accuracy, still below the 93%
of the DNN on hand crafter features, but with a much simpler processing pipeline.


## Preliminaries

### Imports

In [15]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
print(tf.__version__)


%matplotlib inline

import sys
sys.path.append("../..")
import E4525_ML.mnist as mnist

1.11.0


In [16]:
#
from tensorflow.keras.backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)

sess = tf.Session(config=config)

set_session(sess)  # set this TensorFlow session as the default session for Keras

In [17]:
data_dir=r"../../data/UrbanSounds"

In [20]:
file=open(data_dir+"/mfcc.p","rb")
features_all,labels_all=pickle.load(file)
print(features_all.shape,labels_all.shape)

(7895, 20, 173) (7895,)


In [21]:
features,features_test,labels,labels_test=train_test_split(features_all,labels_all,test_size=0.2)
print(features.shape,features_test.shape)


(6316, 20, 173) (1579, 20, 173)


In [22]:
features_train,features_val,labels_train,labels_val=train_test_split(features,labels,test_size=0.2)
print(features_train.shape,labels_train.shape,features_val.shape,labels_val.shape)

(5052, 20, 173) (5052,) (1264, 20, 173) (1264,)


## Convolutional Neural Network

In [100]:
model = keras.Sequential([
    keras.layers.Reshape((20,173,1),input_shape=(20,173)),
    # Convolutional Layers
    keras.layers.Conv2D(16, (2, 4),input_shape=(20,173,1),activation='relu',padding="same"),
     keras.layers.Dropout(0.2), 
    keras.layers.MaxPooling2D(pool_size=(2, 4)),
    
    keras.layers.Conv2D(32, (2, 4),activation='relu',padding="same"),
    keras.layers.MaxPooling2D(pool_size=(2, 4)),
    keras.layers.Dropout(0.3),    
    keras.layers.Conv2D(64, (2, 4),activation='relu',padding="same"),
    keras.layers.MaxPooling2D(pool_size=(2, 4)),
    keras.layers.Dropout(0.3),    
   
    
    # Dense Layers
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [101]:
model.compile(optimizer=keras.optimizers.SGD(),#tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_15 (Reshape)         (None, 20, 173, 1)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 20, 173, 16)       144       
_________________________________________________________________
dropout_50 (Dropout)         (None, 20, 173, 16)       0         
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 10, 43, 16)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 10, 43, 32)        4128      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 5, 10, 32)         0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 5, 10, 32)         0         
__________

In [103]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 3s 526us/step - loss: 3.6675 - acc: 0.1487 - val_loss: 2.1125 - val_acc: 0.2555: 0s - loss: 4.1520 - acc: 0
Epoch 2/200
5052/5052 [==============================] - 1s 245us/step - loss: 2.0896 - acc: 0.2203 - val_loss: 1.9570 - val_acc: 0.3457 0s - loss: 2.1239 - 
Epoch 3/200
5052/5052 [==============================] - 1s 255us/step - loss: 1.9984 - acc: 0.2587 - val_loss: 1.8755 - val_acc: 0.3995
Epoch 4/200
5052/5052 [==============================] - 1s 251us/step - loss: 1.8976 - acc: 0.2989 - val_loss: 1.7630 - val_acc: 0.4359
Epoch 5/200
5052/5052 [==============================] - ETA: 0s - loss: 1.8112 - acc: 0.3337- ETA: 0s - loss: 1.82 - 1s 253us/step - loss: 1.8060 - acc: 0.3359 - val_loss: 1.7022 - val_acc: 0.3956
Epoch 6/200
5052/5052 [==============================] - 1s 250us/step - loss: 1.7517 - acc: 0.3605 - val_loss: 1.7021 - val_acc: 0.4098oss: 1.7511 - acc: 0


5052/5052 [==============================] - 1s 246us/step - loss: 0.7589 - acc: 0.7381 - val_loss: 0.7281 - val_acc: 0.7729
Epoch 109/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.7588 - acc: 0.7350 - val_loss: 0.7167 - val_acc: 0.77690s - loss: 0.7584 - acc
Epoch 110/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.7556 - acc: 0.7423 - val_loss: 0.7161 - val_acc: 0.7777
Epoch 111/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.7612 - acc: 0.7435 - val_loss: 0.7044 - val_acc: 0.7911
Epoch 112/200
5052/5052 [==============================] - ETA: 0s - loss: 0.7469 - acc: 0.741 - 1s 246us/step - loss: 0.7493 - acc: 0.7407 - val_loss: 0.7317 - val_acc: 0.7872
Epoch 113/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.7421 - acc: 0.7506 - val_loss: 0.7285 - val_acc: 0.7603
Epoch 114/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.7282 - acc: 0.7534 - val_loss: 0.695

In [104]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 241us/step - loss: 0.5504 - acc: 0.8108 - val_loss: 0.5558 - val_acc: 0.8188
Epoch 2/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.5271 - acc: 0.8201 - val_loss: 0.5218 - val_acc: 0.8228s: 0.5279 - acc: 0.819
Epoch 3/200
5052/5052 [==============================] - 1s 252us/step - loss: 0.5540 - acc: 0.8104 - val_loss: 0.5528 - val_acc: 0.81880.5458 - acc: 0.81
Epoch 4/200
5052/5052 [==============================] - 1s 249us/step - loss: 0.5244 - acc: 0.8169 - val_loss: 0.5351 - val_acc: 0.8236: 0s - loss: 0.5235 - acc: 0.81
Epoch 5/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.5110 - acc: 0.8274 - val_loss: 0.5537 - val_acc: 0.8038
Epoch 6/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.5124 - acc: 0.8262 - val_loss: 0.7234 - val_acc: 0.7666
Epoch 7/200
5052/5052 [==============================] - 1s

5052/5052 [==============================] - 1s 247us/step - loss: 0.4239 - acc: 0.8529 - val_loss: 0.4671 - val_acc: 0.8434s - loss: 0.4257 - acc: - ETA: 0s - loss: 0.4137 - acc:
Epoch 113/200
5052/5052 [==============================] - 1s 241us/step - loss: 0.4068 - acc: 0.8628 - val_loss: 0.4429 - val_acc: 0.8410
Epoch 114/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.4208 - acc: 0.8523 - val_loss: 0.4580 - val_acc: 0.8457
Epoch 115/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.4151 - acc: 0.8597 - val_loss: 0.4423 - val_acc: 0.8497
Epoch 116/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.4486 - acc: 0.8478 - val_loss: 0.4536 - val_acc: 0.8449
Epoch 117/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.4130 - acc: 0.8614 - val_loss: 0.4530 - val_acc: 0.8386
Epoch 118/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.4042 - acc: 0.8599 - val_loss: 0.4809 - va

In [105]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.3940 - acc: 0.8670 - val_loss: 0.4180 - val_acc: 0.8576
Epoch 2/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.3500 - acc: 0.8806 - val_loss: 0.4293 - val_acc: 0.8473 0s - loss: 0.3447 - 
Epoch 3/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.3546 - acc: 0.8814 - val_loss: 0.4313 - val_acc: 0.8410.3511 - acc: 0.882
Epoch 4/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.3695 - acc: 0.8747 - val_loss: 0.4095 - val_acc: 0.8631
Epoch 5/200
5052/5052 [==============================] - ETA: 0s - loss: 0.3841 - acc: 0.873 - 1s 246us/step - loss: 0.3845 - acc: 0.8737 - val_loss: 0.4342 - val_acc: 0.8528
Epoch 6/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.3596 - acc: 0.8779 - val_loss: 0.4144 - val_acc: 0.8576
Epoch 7/200
5052/5052 [==============================

5052/5052 [==============================] - 1s 245us/step - loss: 0.3153 - acc: 0.8911 - val_loss: 0.3872 - val_acc: 0.8750
Epoch 113/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.3104 - acc: 0.8933 - val_loss: 0.3875 - val_acc: 0.8623
Epoch 114/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.3194 - acc: 0.8915 - val_loss: 0.3866 - val_acc: 0.8703
Epoch 115/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.3036 - acc: 0.9010 - val_loss: 0.3851 - val_acc: 0.8608 - loss: 0.3037 - acc - ETA: 0s - loss: 0.3033 - acc: 0.901
Epoch 116/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.3266 - acc: 0.8897 - val_loss: 0.3983 - val_acc: 0.8631 0s - loss: 0.3192 - acc: 0.89
Epoch 117/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.3176 - acc: 0.8911 - val_loss: 0.4005 - val_acc: 0.8600
Epoch 118/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.3310 - a

Epoch 167/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.2908 - acc: 0.9004 - val_loss: 0.3889 - val_acc: 0.8679
Epoch 168/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2782 - acc: 0.9089 - val_loss: 0.3755 - val_acc: 0.8734
Epoch 169/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2968 - acc: 0.9034 - val_loss: 0.3894 - val_acc: 0.8718
Epoch 170/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2868 - acc: 0.9078 - val_loss: 0.3717 - val_acc: 0.8774s - loss: 0.2662 -
Epoch 171/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2948 - acc: 0.9006 - val_loss: 0.3980 - val_acc: 0.8695
Epoch 172/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2877 - acc: 0.9042 - val_loss: 0.3929 - val_acc: 0.8703
Epoch 173/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.2915 - acc: 0.9028 - val_loss: 0.4104 - val_acc: 0.8639
Epoch 174

In [106]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2584 - acc: 0.9129 - val_loss: 0.3758 - val_acc: 0.8845
Epoch 2/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2656 - acc: 0.9129 - val_loss: 0.3711 - val_acc: 0.8813 0s - loss: 0.2735 -
Epoch 3/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2812 - acc: 0.9084 - val_loss: 0.3924 - val_acc: 0.8750s: 0.2834 - acc: 0.90
Epoch 4/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2705 - acc: 0.9099 - val_loss: 0.3944 - val_acc: 0.8710 0s - loss: 0.2736 - acc: 0 - ETA: 0s - loss: 0.2682 - acc: 0.9
Epoch 5/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.2563 - acc: 0.9119 - val_loss: 0.3846 - val_acc: 0.8774
Epoch 6/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.3014 - acc: 0.9034 - val_loss: 0.3832 - val_acc: 0.8790
Epoch 7/200
5052/5052 [===

Epoch 114/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.2456 - acc: 0.9212 - val_loss: 0.3614 - val_acc: 0.8821
Epoch 115/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2306 - acc: 0.9218 - val_loss: 0.3533 - val_acc: 0.8908
Epoch 116/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2839 - acc: 0.9105 - val_loss: 0.3527 - val_acc: 0.88130s - loss: 0.2828 - acc: 0.91
Epoch 117/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.2422 - acc: 0.9161 - val_loss: 0.3630 - val_acc: 0.8774
Epoch 118/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.2434 - acc: 0.9137 - val_loss: 0.3563 - val_acc: 0.8861
Epoch 119/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2457 - acc: 0.9115 - val_loss: 0.3640 - val_acc: 0.8813
Epoch 120/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.2478 - acc: 0.9216 - val_loss: 0.3535 - val_acc: 0.886

Epoch 167/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2216 - acc: 0.9236 - val_loss: 0.3606 - val_acc: 0.8758
Epoch 168/200
5052/5052 [==============================] - 1s 249us/step - loss: 0.2422 - acc: 0.9210 - val_loss: 0.3524 - val_acc: 0.8900
Epoch 169/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2229 - acc: 0.9238 - val_loss: 0.3756 - val_acc: 0.8845
Epoch 170/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2335 - acc: 0.9202 - val_loss: 0.3787 - val_acc: 0.8797
Epoch 171/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.2528 - acc: 0.9177 - val_loss: 0.3425 - val_acc: 0.8884
Epoch 172/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2374 - acc: 0.9208 - val_loss: 0.3473 - val_acc: 0.8964
Epoch 173/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.2520 - acc: 0.9113 - val_loss: 0.3582 - val_acc: 0.8869
Epoch 174/200
5052/5052 [==

In [107]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.2349 - acc: 0.9198 - val_loss: 0.3869 - val_acc: 0.8797 0s - loss: 0.2213 - acc: 0.9 - ETA: 0s - loss: 0.2286 - acc:
Epoch 2/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2444 - acc: 0.9186 - val_loss: 0.3371 - val_acc: 0.8916
Epoch 3/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.2291 - acc: 0.9262 - val_loss: 0.3551 - val_acc: 0.8837
Epoch 4/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2492 - acc: 0.9192 - val_loss: 0.4335 - val_acc: 0.8418
Epoch 5/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2351 - acc: 0.9206 - val_loss: 0.3442 - val_acc: 0.8877
Epoch 6/200
5052/5052 [==============================] - 1s 241us/step - loss: 0.2328 - acc: 0.9222 - val_loss: 0.3462 - val_acc: 0.8861
Epoch 7/200
5052/5052 [==============================] - 1s 246us/ste

5052/5052 [==============================] - ETA: 0s - loss: 0.2235 - acc: 0.924 - 1s 250us/step - loss: 0.2230 - acc: 0.9242 - val_loss: 0.3352 - val_acc: 0.8908
Epoch 107/200
5052/5052 [==============================] - 1s 261us/step - loss: 0.2050 - acc: 0.9297 - val_loss: 0.3367 - val_acc: 0.8924
Epoch 108/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.2334 - acc: 0.9270 - val_loss: 0.3465 - val_acc: 0.8853
Epoch 109/200
5052/5052 [==============================] - 1s 260us/step - loss: 0.2211 - acc: 0.9299 - val_loss: 0.3273 - val_acc: 0.8892
Epoch 110/200
5052/5052 [==============================] - 1s 261us/step - loss: 0.2452 - acc: 0.9194 - val_loss: 0.3489 - val_acc: 0.8805
Epoch 111/200
5052/5052 [==============================] - 1s 258us/step - loss: 0.2600 - acc: 0.9169 - val_loss: 0.3533 - val_acc: 0.8774
Epoch 112/200
5052/5052 [==============================] - 1s 252us/step - loss: 0.2165 - acc: 0.9268 - val_loss: 0.3705 - val_acc: 0.8718
Epo

Epoch 161/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.2223 - acc: 0.9303 - val_loss: 0.3922 - val_acc: 0.8774
Epoch 162/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2182 - acc: 0.9327 - val_loss: 0.3408 - val_acc: 0.8805
Epoch 163/200
5052/5052 [==============================] - 1s 249us/step - loss: 0.2117 - acc: 0.9335 - val_loss: 0.3326 - val_acc: 0.8790
Epoch 164/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2006 - acc: 0.9331 - val_loss: 0.3657 - val_acc: 0.8663
Epoch 165/200
5052/5052 [==============================] - 1s 249us/step - loss: 0.2218 - acc: 0.9264 - val_loss: 0.3434 - val_acc: 0.8821: 0s - loss: 0.2362 -
Epoch 166/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.2075 - acc: 0.9301 - val_loss: 0.3330 - val_acc: 0.8821
Epoch 167/200
5052/5052 [==============================] - 1s 249us/step - loss: 0.2062 - acc: 0.9297 - val_loss: 0.3541 - val_acc: 0.87970s - lo

In [108]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2136 - acc: 0.9305 - val_loss: 0.3457 - val_acc: 0.8813
Epoch 2/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.2065 - acc: 0.9329 - val_loss: 0.3456 - val_acc: 0.8837 0s - loss: 0.2081 - acc: 0
Epoch 3/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.2044 - acc: 0.9305 - val_loss: 0.3517 - val_acc: 0.8853: 0s - loss: 0.210
Epoch 4/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2155 - acc: 0.9262 - val_loss: 0.3493 - val_acc: 0.8869s - loss: 0.2137 - acc: 0
Epoch 5/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2094 - acc: 0.9299 - val_loss: 0.3344 - val_acc: 0.8790
Epoch 6/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1882 - acc: 0.9345 - val_loss: 0.3303 - val_acc: 0.8869
Epoch 7/200
5052/5052 [==============================] - 1s 

5052/5052 [==============================] - 1s 243us/step - loss: 0.2075 - acc: 0.9283 - val_loss: 0.3306 - val_acc: 0.8869
Epoch 113/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.1986 - acc: 0.9351 - val_loss: 0.3844 - val_acc: 0.8797
Epoch 114/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2074 - acc: 0.9323 - val_loss: 0.3773 - val_acc: 0.8766- loss: 0.2039 - acc: 0.933 - ETA: 0s - loss: 0.2003 - acc:
Epoch 115/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2195 - acc: 0.9295 - val_loss: 0.3754 - val_acc: 0.8813
Epoch 116/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1937 - acc: 0.9369 - val_loss: 0.3422 - val_acc: 0.8837
Epoch 117/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1877 - acc: 0.9414 - val_loss: 0.3466 - val_acc: 0.8805
Epoch 118/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1921 - acc: 0.9390 - val_loss: 0.3628 

Epoch 166/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1929 - acc: 0.9392 - val_loss: 0.3078 - val_acc: 0.8987
Epoch 167/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.1800 - acc: 0.9406 - val_loss: 0.3180 - val_acc: 0.8932
Epoch 168/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1933 - acc: 0.9384 - val_loss: 0.3099 - val_acc: 0.9003
Epoch 169/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2221 - acc: 0.9270 - val_loss: 0.3196 - val_acc: 0.8972
Epoch 170/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1951 - acc: 0.9341 - val_loss: 0.3149 - val_acc: 0.8972
Epoch 171/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1885 - acc: 0.9402 - val_loss: 0.3493 - val_acc: 0.8845
Epoch 172/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1714 - acc: 0.9436 - val_loss: 0.3357 - val_acc: 0.8900
Epoch 173/200
5052/5052 [==

In [109]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1755 - acc: 0.9430 - val_loss: 0.3242 - val_acc: 0.8987 0s - loss: 0.1842 - acc:
Epoch 2/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.2135 - acc: 0.9268 - val_loss: 0.3437 - val_acc: 0.8829
Epoch 3/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1973 - acc: 0.9335 - val_loss: 0.3402 - val_acc: 0.8900
Epoch 4/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.2048 - acc: 0.9293 - val_loss: 0.3265 - val_acc: 0.8829oss: 0.2144 - acc: 0.928 - ETA: 0s - loss: 0.2087 - acc: 0.9 - ETA: 0s - loss: 0.2069 - acc: 0.929 - ETA: 0s - loss: 0.2069 - acc: 0.928
Epoch 5/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1943 - acc: 0.9355 - val_loss: 0.3238 - val_acc: 0.8948
Epoch 6/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.2112 - acc: 0.9333 - val_

5052/5052 [==============================] - 1s 252us/step - loss: 0.1905 - acc: 0.9428 - val_loss: 0.3168 - val_acc: 0.8908
Epoch 114/200
5052/5052 [==============================] - 1s 255us/step - loss: 0.1870 - acc: 0.9406 - val_loss: 0.3149 - val_acc: 0.9011
Epoch 115/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1788 - acc: 0.9424 - val_loss: 0.3187 - val_acc: 0.8892
Epoch 116/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1900 - acc: 0.9351 - val_loss: 0.3443 - val_acc: 0.8821
Epoch 117/200
5052/5052 [==============================] - 1s 252us/step - loss: 0.1865 - acc: 0.9418 - val_loss: 0.3151 - val_acc: 0.9011
Epoch 118/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.2041 - acc: 0.9331 - val_loss: 0.3103 - val_acc: 0.9011
Epoch 119/200
5052/5052 [==============================] - 1s 249us/step - loss: 0.1906 - acc: 0.9375 - val_loss: 0.3335 - val_acc: 0.8861: 0s - loss: 0.2041 - 
Epoch 120/200
5052/

In [110]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.1416 - acc: 0.9565 - val_loss: 0.3303 - val_acc: 0.8956
Epoch 2/200
5052/5052 [==============================] - 1s 260us/step - loss: 0.1784 - acc: 0.9428 - val_loss: 0.3464 - val_acc: 0.8877
Epoch 3/200
5052/5052 [==============================] - 1s 252us/step - loss: 0.1751 - acc: 0.9410 - val_loss: 0.3909 - val_acc: 0.8758
Epoch 4/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.1754 - acc: 0.9428 - val_loss: 0.3254 - val_acc: 0.8774
Epoch 5/200
5052/5052 [==============================] - 1s 252us/step - loss: 0.1711 - acc: 0.9446 - val_loss: 0.3623 - val_acc: 0.8892
Epoch 6/200
5052/5052 [==============================] - 1s 252us/step - loss: 0.1820 - acc: 0.9426 - val_loss: 0.3148 - val_acc: 0.8979
Epoch 7/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.2073 - acc: 0.9331 - val_loss: 0.3395 - val_acc: 

Epoch 59/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1860 - acc: 0.9380 - val_loss: 0.3266 - val_acc: 0.8877
Epoch 60/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1590 - acc: 0.9505 - val_loss: 0.3283 - val_acc: 0.8908
Epoch 61/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1724 - acc: 0.9420 - val_loss: 0.3152 - val_acc: 0.8900 0s - loss: 0.1756 - acc: 0.9
Epoch 62/200
5052/5052 [==============================] - 1s 241us/step - loss: 0.1709 - acc: 0.9483 - val_loss: 0.3356 - val_acc: 0.8813
Epoch 63/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1705 - acc: 0.9446 - val_loss: 0.3458 - val_acc: 0.8861
Epoch 64/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1801 - acc: 0.9432 - val_loss: 0.3246 - val_acc: 0.8869
Epoch 65/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1804 - acc: 0.9454 - val_loss: 0.3199 - val_acc: 0.9003
Epoch

Epoch 116/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1662 - acc: 0.9489 - val_loss: 0.3624 - val_acc: 0.8766
Epoch 117/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1781 - acc: 0.9454 - val_loss: 0.3356 - val_acc: 0.8869
Epoch 118/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1880 - acc: 0.9353 - val_loss: 0.3141 - val_acc: 0.8924
Epoch 119/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.1723 - acc: 0.9434 - val_loss: 0.3342 - val_acc: 0.8845s: 0.1729 - acc: 0
Epoch 120/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1782 - acc: 0.9414 - val_loss: 0.3208 - val_acc: 0.8932
Epoch 121/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1658 - acc: 0.9491 - val_loss: 0.3311 - val_acc: 0.8837
Epoch 122/200
5052/5052 [==============================] - 1s 241us/step - loss: 0.2604 - acc: 0.9283 - val_loss: 0.3310 - val_acc: 0.8908
Epoch 123

In [111]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1622 - acc: 0.9483 - val_loss: 0.3072 - val_acc: 0.8964
Epoch 2/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1785 - acc: 0.9426 - val_loss: 0.2971 - val_acc: 0.9019
Epoch 3/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1618 - acc: 0.9499 - val_loss: 0.3180 - val_acc: 0.8987
Epoch 4/200
5052/5052 [==============================] - ETA: 0s - loss: 0.1618 - acc: 0.9464- ETA: 0s - loss: 0.1684 - acc: 0.945 - ETA: 0s - loss: 0.1744 - acc:  - 1s 244us/step - loss: 0.1600 - acc: 0.9470 - val_loss: 0.3295 - val_acc: 0.8964
Epoch 5/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1686 - acc: 0.9485 - val_loss: 0.3170 - val_acc: 0.8956
Epoch 6/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1639 - acc: 0.9475 - val_loss: 0.3627 - val_acc: 0.8884
Epoch 7/200
5052/5052

Epoch 113/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1495 - acc: 0.9539 - val_loss: 0.3184 - val_acc: 0.8924
Epoch 114/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1715 - acc: 0.9464 - val_loss: 0.3221 - val_acc: 0.8916
Epoch 115/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1705 - acc: 0.9442 - val_loss: 0.3293 - val_acc: 0.8908
Epoch 116/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1662 - acc: 0.9471 - val_loss: 0.3529 - val_acc: 0.8861
Epoch 117/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1627 - acc: 0.9473 - val_loss: 0.3152 - val_acc: 0.8940 loss: 0.1689 - acc: 0.9
Epoch 118/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1473 - acc: 0.9523 - val_loss: 0.3282 - val_acc: 0.8940
Epoch 119/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1677 - acc: 0.9470 - val_loss: 0.3250 - val_acc: 0.8900
Epo

In [112]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1582 - acc: 0.9475 - val_loss: 0.3376 - val_acc: 0.8924
Epoch 2/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1560 - acc: 0.9481 - val_loss: 0.3407 - val_acc: 0.8861: 0s - loss: 0.1929 - acc:  - ETA: 0s - loss: 0.1711 - acc: 
Epoch 3/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1301 - acc: 0.9557 - val_loss: 0.3251 - val_acc: 0.8884ss: 0.1317 - acc: 0.95 - ETA: 0s - loss: 0.1286 - acc: 0.95
Epoch 4/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1833 - acc: 0.9464 - val_loss: 0.3942 - val_acc: 0.8805
Epoch 5/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1659 - acc: 0.9468 - val_loss: 0.3046 - val_acc: 0.8987
Epoch 6/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1540 - acc: 0.9501 - val_loss: 0.3198 - val_acc: 0.9027
Epoch 7/200

5052/5052 [==============================] - 1s 245us/step - loss: 0.1540 - acc: 0.9497 - val_loss: 0.3213 - val_acc: 0.8908
Epoch 111/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1525 - acc: 0.9507 - val_loss: 0.3312 - val_acc: 0.8853
Epoch 112/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1588 - acc: 0.9495 - val_loss: 0.3153 - val_acc: 0.8987 loss: 0.1673 - acc: 0.94 - ETA: 0s - loss: 0.1642 - acc: 0.949 - ETA: 0s - loss: 0.1633 - acc: 0.948 - ETA: 0s - loss: 0.1621 - acc: 0.94 - ETA: 0s - loss: 0.1624 - acc: 0.9
Epoch 113/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1721 - acc: 0.9503 - val_loss: 0.3171 - val_acc: 0.8916
Epoch 114/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1553 - acc: 0.9491 - val_loss: 0.3175 - val_acc: 0.8884loss: 0.1554 - a
Epoch 115/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1556 - acc: 0.9489 - val_loss: 0.3257 - val_ac

In [113]:
model.fit(features_train, labels_train, epochs=200,validation_data=(features_val,labels_val))

Train on 5052 samples, validate on 1264 samples
Epoch 1/200
5052/5052 [==============================] - ETA: 0s - loss: 0.1431 - acc: 0.957 - 1s 247us/step - loss: 0.1426 - acc: 0.9574 - val_loss: 0.3342 - val_acc: 0.8900
Epoch 2/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1577 - acc: 0.9497 - val_loss: 0.3388 - val_acc: 0.8813
Epoch 3/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1337 - acc: 0.9519 - val_loss: 0.3191 - val_acc: 0.8987
Epoch 4/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1465 - acc: 0.9533 - val_loss: 0.3445 - val_acc: 0.8916
Epoch 5/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.1396 - acc: 0.9567 - val_loss: 0.3420 - val_acc: 0.8924
Epoch 6/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1483 - acc: 0.9513 - val_loss: 0.3176 - val_acc: 0.8869
Epoch 7/200
5052/5052 [==============================] - 1s 242us/step - loss: 0.1650 - acc:

5052/5052 [==============================] - 1s 245us/step - loss: 0.1572 - acc: 0.9511 - val_loss: 0.3417 - val_acc: 0.8884 0s - loss: 0.1525 -
Epoch 110/200
5052/5052 [==============================] - 1s 245us/step - loss: 0.1554 - acc: 0.9533 - val_loss: 0.2997 - val_acc: 0.8900
Epoch 111/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1499 - acc: 0.9539 - val_loss: 0.3385 - val_acc: 0.8845
Epoch 112/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1535 - acc: 0.9493 - val_loss: 0.3108 - val_acc: 0.8940
Epoch 113/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1385 - acc: 0.9541 - val_loss: 0.3550 - val_acc: 0.8853
Epoch 114/200
5052/5052 [==============================] - 1s 246us/step - loss: 0.1618 - acc: 0.9470 - val_loss: 0.3299 - val_acc: 0.8924
Epoch 115/200
5052/5052 [==============================] - ETA: 0s - loss: 0.1352 - acc: 0.9530- ETA: 1s - loss: 0.1 - 1s 247us/step - loss: 0.1351 - acc: 0.953

Epoch 163/200
5052/5052 [==============================] - 1s 248us/step - loss: 0.1447 - acc: 0.9495 - val_loss: 0.3247 - val_acc: 0.8892s - loss: 0.1188 - acc: 0.955 - ETA: 0s - loss: 0.1241 - acc: - ETA: 0s - loss: 0.1440 - acc: 0.9
Epoch 164/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1422 - acc: 0.9561 - val_loss: 0.3314 - val_acc: 0.8877
Epoch 165/200
5052/5052 [==============================] - 1s 247us/step - loss: 0.1575 - acc: 0.9481 - val_loss: 0.3175 - val_acc: 0.8956
Epoch 166/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1425 - acc: 0.9527 - val_loss: 0.3190 - val_acc: 0.8900
Epoch 167/200
5052/5052 [==============================] - 1s 243us/step - loss: 0.1682 - acc: 0.9519 - val_loss: 0.3171 - val_acc: 0.8877
Epoch 168/200
5052/5052 [==============================] - 1s 244us/step - loss: 0.1334 - acc: 0.9568 - val_loss: 0.3341 - val_acc: 0.8964
Epoch 169/200
5052/5052 [==============================] - 1s 245us/s

## Test

In [114]:
model.fit(features, labels, epochs=2000,validation_data=(features_test,labels_test))

Train on 6316 samples, validate on 1579 samples
Epoch 1/2000
6316/6316 [==============================] - 2s 254us/step - loss: 0.2580 - acc: 0.9297 - val_loss: 0.3874 - val_acc: 0.8809
Epoch 2/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.2521 - acc: 0.9307 - val_loss: 0.3572 - val_acc: 0.8828: 0s - loss: 0.265 - ETA: 0s - loss: 0.2524 - acc: 0.930
Epoch 3/2000
6316/6316 [==============================] - 2s 242us/step - loss: 0.2584 - acc: 0.9223 - val_loss: 0.4044 - val_acc: 0.8632
Epoch 4/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.2659 - acc: 0.9178 - val_loss: 0.3373 - val_acc: 0.8879
Epoch 5/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.2362 - acc: 0.9269 - val_loss: 0.3329 - val_acc: 0.8923: 0s - loss: 0.2
Epoch 6/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.2152 - acc: 0.9329 - val_loss: 0.3360 - val_acc: 0.8898
Epoch 7/2000
6316/6316 [============================

Epoch 57/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.2001 - acc: 0.9367 - val_loss: 0.3388 - val_acc: 0.8847
Epoch 58/2000
6316/6316 [==============================] - 2s 248us/step - loss: 0.1893 - acc: 0.9378 - val_loss: 0.3369 - val_acc: 0.8885ss: 0.1945 - acc: 0.93 - ETA: 0s - loss: 0.1942 - acc: 0
Epoch 59/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.2212 - acc: 0.9300 - val_loss: 0.3265 - val_acc: 0.8866
Epoch 60/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1896 - acc: 0.9381 - val_loss: 0.5614 - val_acc: 0.8493
Epoch 61/2000
6316/6316 [==============================] - 2s 248us/step - loss: 0.2061 - acc: 0.9367 - val_loss: 0.3171 - val_acc: 0.8942
Epoch 62/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.2075 - acc: 0.9295 - val_loss: 0.3242 - val_acc: 0.8968- loss: 0.2229 -
Epoch 63/2000
6316/6316 [==============================] - 2s 249us/step - loss: 0.1874 - acc:

6316/6316 [==============================] - 2s 246us/step - loss: 0.1781 - acc: 0.9438 - val_loss: 0.3164 - val_acc: 0.8968
Epoch 164/2000
6316/6316 [==============================] - 2s 247us/step - loss: 0.1811 - acc: 0.9402 - val_loss: 0.3151 - val_acc: 0.8974 1s - loss: 0.1683 - acc: - ETA: 0s - loss: 0.1747 - a - ETA: 0s - loss: 0.1807 - acc: 0.940
Epoch 165/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1799 - acc: 0.9395 - val_loss: 0.3391 - val_acc: 0.8936
Epoch 166/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1900 - acc: 0.9387 - val_loss: 0.2998 - val_acc: 0.9063ss: 0.1940 - acc: 0.9
Epoch 167/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1906 - acc: 0.9383 - val_loss: 0.3159 - val_acc: 0.8987 loss: 0.1920 - acc: 0 - ETA: 0s - loss: 0.1928 - acc: 0.93 - ETA: 0s - loss: 0.1898 - acc: 0.9
Epoch 168/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1769 - acc: 0.9414 - va

6316/6316 [==============================] - 2s 243us/step - loss: 0.1721 - acc: 0.9447 - val_loss: 0.3215 - val_acc: 0.90180s - loss: 0.1705 - a
Epoch 271/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1891 - acc: 0.9413 - val_loss: 0.3402 - val_acc: 0.8955
Epoch 272/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1881 - acc: 0.9400 - val_loss: 0.3188 - val_acc: 0.8930
Epoch 273/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1711 - acc: 0.9425 - val_loss: 0.3174 - val_acc: 0.8949
Epoch 274/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1754 - acc: 0.9457 - val_loss: 0.3353 - val_acc: 0.8930
Epoch 275/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1751 - acc: 0.9441 - val_loss: 0.3242 - val_acc: 0.8999
Epoch 276/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1813 - acc: 0.9373 - val_loss: 0.3243 - val_acc: 0.8974s - loss: 0.183

Epoch 325/2000
6316/6316 [==============================] - ETA: 0s - loss: 0.1733 - acc: 0.945 - 2s 245us/step - loss: 0.1725 - acc: 0.9452 - val_loss: 0.3269 - val_acc: 0.9012
Epoch 326/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.2024 - acc: 0.9373 - val_loss: 0.3105 - val_acc: 0.9050
Epoch 327/2000
6316/6316 [==============================] - ETA: 0s - loss: 0.1980 - acc: 0.9378- ETA: 0s - loss: 0.181 - 2s 242us/step - loss: 0.1978 - acc: 0.9379 - val_loss: 0.3258 - val_acc: 0.8974
Epoch 328/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1636 - acc: 0.9459 - val_loss: 0.3350 - val_acc: 0.8987
Epoch 329/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1826 - acc: 0.9432 - val_loss: 0.3206 - val_acc: 0.8993
Epoch 330/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1669 - acc: 0.9451 - val_loss: 0.3223 - val_acc: 0.9044
Epoch 331/2000
6316/6316 [==============================] - 

...

6316/6316 [==============================] - 2s 244us/step - loss: 0.1563 - acc: 0.9519 - val_loss: 0.3134 - val_acc: 0.8993
Epoch 1555/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1403 - acc: 0.9508 - val_loss: 0.3144 - val_acc: 0.9063
Epoch 1556/2000
6316/6316 [==============================] - 2s 241us/step - loss: 0.1451 - acc: 0.9569 - val_loss: 0.3171 - val_acc: 0.9088
Epoch 1557/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1544 - acc: 0.9516 - val_loss: 0.3024 - val_acc: 0.9145
Epoch 1558/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1404 - acc: 0.9554 - val_loss: 0.3203 - val_acc: 0.9075
Epoch 1559/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1492 - acc: 0.9522 - val_loss: 0.3300 - val_acc: 0.8993
Epoch 1560/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1442 - acc: 0.9552 - val_loss: 0.3014 - val_acc: 0.9145
Epoch 1561/2000
6316/6316 [==

6316/6316 [==============================] - ETA: 0s - loss: 0.1585 - acc: 0.9511- ETA: 0s - loss: 0.1588 - acc: 0. - 2s 243us/step - loss: 0.1561 - acc: 0.9519 - val_loss: 0.3061 - val_acc: 0.9094
Epoch 1662/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1367 - acc: 0.9547 - val_loss: 0.3131 - val_acc: 0.9056
Epoch 1663/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1460 - acc: 0.9544 - val_loss: 0.3007 - val_acc: 0.91260s - loss: 0.1350 - acc: 0 - ETA: 0s - loss: 0.1446 - ac
Epoch 1664/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1548 - acc: 0.9542 - val_loss: 0.2948 - val_acc: 0.9088 loss: 0.1523 
Epoch 1665/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1499 - acc: 0.9538 - val_loss: 0.3048 - val_acc: 0.9037- loss: 0.1446 - acc: 0.951 - ETA: 0s - loss: 0.1424 - 
Epoch 1666/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1627 - acc: 0.9531 - val_los

Epoch 1713/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1486 - acc: 0.9558 - val_loss: 0.2858 - val_acc: 0.9107 - acc: 0.955
Epoch 1714/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1544 - acc: 0.9552 - val_loss: 0.3222 - val_acc: 0.8980
Epoch 1715/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1286 - acc: 0.9560 - val_loss: 0.2976 - val_acc: 0.9075.1290 - acc: 0.9 - ETA: 0s - loss: 0.1288 - acc: 0.956
Epoch 1716/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1405 - acc: 0.9536 - val_loss: 0.2844 - val_acc: 0.9101
Epoch 1717/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1398 - acc: 0.9576 - val_loss: 0.2876 - val_acc: 0.9132
Epoch 1718/2000
6316/6316 [==============================] - ETA: 0s - loss: 0.1458 - acc: 0.9544- ETA: 0s - loss: 0.1460 - - 2s 243us/step - loss: 0.1434 - acc: 0.9554 - val_loss: 0.3027 - val_acc: 0.9056
Epoch 1719/2000
6316/6

6316/6316 [==============================] - 2s 242us/step - loss: 0.1534 - acc: 0.9522 - val_loss: 0.2965 - val_acc: 0.9132loss: 0.1518 - ac
Epoch 1817/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1369 - acc: 0.9569 - val_loss: 0.3026 - val_acc: 0.9107
Epoch 1818/2000
6316/6316 [==============================] - ETA: 0s - loss: 0.1365 - acc: 0.9548- ETA: 1s - loss: 0.140 - ETA: 0s - loss: 0.1351 - acc: 0.9 - 2s 244us/step - loss: 0.1339 - acc: 0.9557 - val_loss: 0.2956 - val_acc: 0.9177
Epoch 1819/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1421 - acc: 0.9536 - val_loss: 0.3177 - val_acc: 0.9044 - loss: 0.1443 - acc:  - ETA: 0s - loss: 0.1405 - a
Epoch 1820/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1421 - acc: 0.9520 - val_loss: 0.3712 - val_acc: 0.8936
Epoch 1821/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1212 - acc: 0.9596 - val_loss: 0.3099 - val_acc: 0.9025
Epo

Epoch 1922/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1422 - acc: 0.9539 - val_loss: 0.2938 - val_acc: 0.9082
Epoch 1923/2000
6316/6316 [==============================] - 2s 242us/step - loss: 0.1327 - acc: 0.9565 - val_loss: 0.2991 - val_acc: 0.9075
Epoch 1924/2000
6316/6316 [==============================] - 2s 248us/step - loss: 0.1436 - acc: 0.9550 - val_loss: 0.3139 - val_acc: 0.9063
Epoch 1925/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1319 - acc: 0.9596 - val_loss: 0.3172 - val_acc: 0.9126
Epoch 1926/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1302 - acc: 0.9582 - val_loss: 0.3170 - val_acc: 0.9044
Epoch 1927/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1334 - acc: 0.9582 - val_loss: 0.3130 - val_acc: 0.9107
Epoch 1928/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1549 - acc: 0.9525 - val_loss: 0.2999 - val_acc: 0.9044 - loss: 0.160

In [115]:
model.fit(features, labels, epochs=2000,validation_data=(features_test,labels_test))

Train on 6316 samples, validate on 1579 samples
Epoch 1/2000
6316/6316 [==============================] - 2s 247us/step - loss: 0.1402 - acc: 0.9544 - val_loss: 0.3159 - val_acc: 0.9050
Epoch 2/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1419 - acc: 0.9546 - val_loss: 0.3062 - val_acc: 0.90690s - loss: 0.1330 -
Epoch 3/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1446 - acc: 0.9547 - val_loss: 0.3057 - val_acc: 0.9088: 0s - loss: 0.1423 - acc: 0.
Epoch 4/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1365 - acc: 0.9585 - val_loss: 0.3117 - val_acc: 0.9069s - loss: 0.1235 -  - ETA: 0s - loss: 0.1399 - acc: 
Epoch 5/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1563 - acc: 0.9525 - val_loss: 0.3182 - val_acc: 0.9050oss: 0.1643 -
Epoch 6/2000
6316/6316 [==============================] - 2s 242us/step - loss: 0.1489 - acc: 0.9528 - val_loss: 0.3085 - val_acc: 0.9082: 0.1399 - a

Epoch 57/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1424 - acc: 0.9554 - val_loss: 0.3288 - val_acc: 0.9101
Epoch 58/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1475 - acc: 0.9522 - val_loss: 0.3238 - val_acc: 0.9006
Epoch 59/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1210 - acc: 0.9618 - val_loss: 0.3259 - val_acc: 0.8974
Epoch 60/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1436 - acc: 0.9568 - val_loss: 0.3279 - val_acc: 0.8987
Epoch 61/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1508 - acc: 0.9531 - val_loss: 0.3047 - val_acc: 0.9069
Epoch 62/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1299 - acc: 0.9588 - val_loss: 0.3085 - val_acc: 0.9094
Epoch 63/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1487 - acc: 0.9533 - val_loss: 0.3002 - val_acc: 0.9050
Epoch 64/2000
6316/6316 [==

6316/6316 [==============================] - 2s 243us/step - loss: 0.1383 - acc: 0.9565 - val_loss: 0.3003 - val_acc: 0.9082ss: 0.1395 - 
Epoch 165/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1419 - acc: 0.9584 - val_loss: 0.2969 - val_acc: 0.9025
Epoch 166/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1353 - acc: 0.9569 - val_loss: 0.2757 - val_acc: 0.9170
Epoch 167/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1436 - acc: 0.9533 - val_loss: 0.2944 - val_acc: 0.9069
Epoch 168/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1426 - acc: 0.9544 - val_loss: 0.3018 - val_acc: 0.9145
Epoch 169/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1390 - acc: 0.9579 - val_loss: 0.3032 - val_acc: 0.9170ss: 0.1430 - acc: 0.954 - ETA: 0s - loss: 0.1384 - a
Epoch 170/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1340 - acc: 0.9573 - val_l

6316/6316 [==============================] - 2s 244us/step - loss: 0.1167 - acc: 0.9644 - val_loss: 0.3134 - val_acc: 0.9050: 0s - loss: 0.1133 - acc: 0. - ETA: 0s - loss: 0.1192 
Epoch 270/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1299 - acc: 0.9576 - val_loss: 0.3098 - val_acc: 0.9088
Epoch 271/2000
6316/6316 [==============================] - 2s 242us/step - loss: 0.1489 - acc: 0.9561 - val_loss: 0.3068 - val_acc: 0.9075
Epoch 272/2000
6316/6316 [==============================] - ETA: 0s - loss: 0.1429 - acc: 0.9526- ETA: 0s - loss: 0.1332 - acc: 0.95 - ETA: 0s - loss: 0.1377 - acc: - 2s 245us/step - loss: 0.1413 - acc: 0.9535 - val_loss: 0.2960 - val_acc: 0.9113
Epoch 273/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1441 - acc: 0.9554 - val_loss: 0.3028 - val_acc: 0.9082
Epoch 274/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1395 - acc: 0.9587 - val_loss: 0.2992 - val_acc: 0.9094
Epoch 275/200

6316/6316 [==============================] - 2s 242us/step - loss: 0.1286 - acc: 0.9593 - val_loss: 0.3124 - val_acc: 0.9082
Epoch 376/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1173 - acc: 0.9623 - val_loss: 0.3301 - val_acc: 0.9069
Epoch 377/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1422 - acc: 0.9542 - val_loss: 0.3215 - val_acc: 0.9120- loss: 0.1450 - acc: 0.95
Epoch 378/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1356 - acc: 0.9557 - val_loss: 0.3031 - val_acc: 0.9069 0s - loss: 0.1255 - acc: 0.9 - ETA: 0s - loss: 0.1276 - acc: - ETA: 0s - loss: 0.1350 - acc: 0.956
Epoch 379/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1238 - acc: 0.9593 - val_loss: 0.3194 - val_acc: 0.9056
Epoch 380/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1227 - acc: 0.9609 - val_loss: 0.3078 - val_acc: 0.9094
Epoch 381/2000
6316/6316 [========================

...

6316/6316 [==============================] - 2s 241us/step - loss: 0.1254 - acc: 0.9593 - val_loss: 0.3039 - val_acc: 0.9164 1s - loss: 0.
Epoch 1586/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1369 - acc: 0.9588 - val_loss: 0.3027 - val_acc: 0.9151
Epoch 1587/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1270 - acc: 0.9596 - val_loss: 0.3168 - val_acc: 0.9107
Epoch 1588/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1169 - acc: 0.9618 - val_loss: 0.3007 - val_acc: 0.9151
Epoch 1589/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1310 - acc: 0.9598 - val_loss: 0.2851 - val_acc: 0.9107
Epoch 1590/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1277 - acc: 0.9595 - val_loss: 0.2878 - val_acc: 0.9151
Epoch 1591/2000
6316/6316 [==============================] - 2s 242us/step - loss: 0.1307 - acc: 0.9623 - val_loss: 0.3247 - val_acc: 0.9025
Epoch 1592/2000

6316/6316 [==============================] - 2s 245us/step - loss: 0.1181 - acc: 0.9622 - val_loss: 0.3182 - val_acc: 0.9202
Epoch 1641/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1254 - acc: 0.9636 - val_loss: 0.2982 - val_acc: 0.9126
Epoch 1642/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1177 - acc: 0.9592 - val_loss: 0.3041 - val_acc: 0.9145: 0s - loss: 0.10
Epoch 1643/2000
6316/6316 [==============================] - ETA: 0s - loss: 0.1208 - acc: 0.961 - 2s 245us/step - loss: 0.1206 - acc: 0.9612 - val_loss: 0.2901 - val_acc: 0.9120
Epoch 1644/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1174 - acc: 0.9641 - val_loss: 0.3013 - val_acc: 0.9120
Epoch 1645/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1218 - acc: 0.9617 - val_loss: 0.3046 - val_acc: 0.9113
Epoch 1646/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1339 - acc: 0.9601 - val_loss:

6316/6316 [==============================] - 2s 245us/step - loss: 0.1191 - acc: 0.9603 - val_loss: 0.2935 - val_acc: 0.9044
Epoch 1750/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1405 - acc: 0.9585 - val_loss: 0.3108 - val_acc: 0.9107
Epoch 1751/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1288 - acc: 0.9614 - val_loss: 0.2937 - val_acc: 0.9088
Epoch 1752/2000
6316/6316 [==============================] - 2s 242us/step - loss: 0.1272 - acc: 0.9620 - val_loss: 0.3136 - val_acc: 0.9031 - loss: 0.1252 - acc: 0.
Epoch 1753/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1245 - acc: 0.9625 - val_loss: 0.3102 - val_acc: 0.9050
Epoch 1754/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1274 - acc: 0.9588 - val_loss: 0.3048 - val_acc: 0.9050
Epoch 1755/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1122 - acc: 0.9666 - val_loss: 0.2883 - val_acc: 0.9101
Epoc

Epoch 1804/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1336 - acc: 0.9576 - val_loss: 0.2758 - val_acc: 0.9158
Epoch 1805/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1200 - acc: 0.9630 - val_loss: 0.2817 - val_acc: 0.9139s - loss: 0.1249 - acc:
Epoch 1806/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1198 - acc: 0.9623 - val_loss: 0.2756 - val_acc: 0.9164
Epoch 1807/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1233 - acc: 0.9615 - val_loss: 0.2994 - val_acc: 0.9208: 0s - loss: 0.1149 - 
Epoch 1808/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1368 - acc: 0.9618 - val_loss: 0.2771 - val_acc: 0.9132
Epoch 1809/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1275 - acc: 0.9598 - val_loss: 0.2861 - val_acc: 0.9145
Epoch 1810/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1140 - acc: 0.9649 - val

6316/6316 [==============================] - 2s 242us/step - loss: 0.1552 - acc: 0.9536 - val_loss: 0.3139 - val_acc: 0.8974loss: 0.1581 - acc:
Epoch 1909/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1349 - acc: 0.9569 - val_loss: 0.2993 - val_acc: 0.9056
Epoch 1910/2000
6316/6316 [==============================] - 2s 245us/step - loss: 0.1388 - acc: 0.9566 - val_loss: 0.3036 - val_acc: 0.9139
Epoch 1911/2000
6316/6316 [==============================] - 2s 246us/step - loss: 0.1184 - acc: 0.9623 - val_loss: 0.3197 - val_acc: 0.9101: 0s - loss: 0.1165 - acc: 0.959 - ETA: 0s - loss: 0.115
Epoch 1912/2000
6316/6316 [==============================] - 2s 244us/step - loss: 0.1154 - acc: 0.9604 - val_loss: 0.3261 - val_acc: 0.9018 0.1182 - acc: 0.9
Epoch 1913/2000
6316/6316 [==============================] - 2s 243us/step - loss: 0.1322 - acc: 0.9615 - val_loss: 0.2989 - val_acc: 0.9082
Epoch 1914/2000
6316/6316 [==============================] - 2s 245us/step - 